### Test Check PySpark

In [3]:

import os 
os.environ['PYSPARK_PYTHON'] = '/usr/bin/python3.7'
os.environ['PYSPARK_DRIVER_PYTHON'] = '/usr/bin/python3.7'

In [ ]:
!wget https://gitlab.com/juanmanoel/dataset-atp-big-data-frameworks/-/raw/master/ocorrencias_criminais_corrigida.csv

## Analise Exploratoria ATP Dataset



## Dicionario de Dados

O dicionário de dados deste dataset é declarado a seguir:
Campo 	

| Campo | DESCRIÇÃO |
|-------|-----------|
| Dia   |  Dia da ocorrência     |
| Mes   |  Mes da ocorrência     |
| Ano   |  Ano da ocorrência     |
| Bloco |Região da ocorrência  |   Região da ocorrência 
| Tipo  |  Tipo da ocorrência criminal 
| Descrição |   Breve descrição da ocorrência 
| Descrição da localização | Descrição da localização da ocorrência (rua, por exemplo) 
| Latitude |   Localização da ocorrência 
| Longitude |  Localização da ocorrência 

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("ATP Analise Exploratoria").config("spark.some.config.option","some-value").getOrCreate()

In [4]:
from pyspark import SparkContext
sc = SparkContext

In [17]:
# USE APENAS QUANDO NÃO TIVER RODANDO NO SERVIDOR LOCAL
#!wget -c  https://gitlab.com/juanmanoel/dataset-atp-big-data-frameworks/-/raw/master/ocorrencias_criminais_corrigida.csv

In [20]:
df = spark.read.option("header",True).csv("/home2/ead2020/SEM2/juan.manoel/ocorrencias_criminais_corrigida.csv")

##### Analise Dataframe entendimento da tabela

In [8]:
df.show()

+---+---+----+--------------------+--------------------+--------------------+------------------------+------------+-------------+
|Dia|Mes| Ano|               Bloco|                Tipo|           Descricao|Descrição_da_localizacao|    Latitude|    Longitude|
+---+---+----+--------------------+--------------------+--------------------+------------------------+------------+-------------+
| 18|  3|2015|     047XX W OHIO ST|             BATTERY| AGGRAVATED: HANDGUN|                  STREET|41.891398861|-87.744384567|
| 18|  3|2015|066XX S MARSHFIEL...|       OTHER OFFENSE|    PAROLE VIOLATION|                  STREET|41.773371528|-87.665319468|
| 18|  3|2015|044XX S LAKE PARK...|             BATTERY|DOMESTIC BATTERY ...|               APARTMENT| 41.81386068|-87.596642837|
| 18|  3|2015|051XX S MICHIGAN AVE|             BATTERY|              SIMPLE|               APARTMENT|41.800802415|-87.622619343|
| 18|  3|2015|    047XX W ADAMS ST|             ROBBERY|      ARMED: HANDGUN|             

In [9]:
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import col, avg
import pyspark.sql.functions as func
from pyspark.sql.functions import countDistinct, avg, stddev

## Quantidade de crimes por ano?
----

* Usando Select para formatar a data da coluna Ano e definindo meu alias para Ano
* Em seguida passei o agrupamento por Ano 
* contei os ocorridos por ano.

In [10]:
df.select(F.date_format('Ano','dd/MM/yyyy').alias('Ano')).groupby('Ano').count().show() 

+----------+-------+
|       Ano|  count|
+----------+-------+
|01/01/2011| 359474|
|01/01/2004| 482572|
|01/01/2006| 451114|
|01/01/2013| 308550|
|01/01/2002| 478166|
|01/01/2001| 497358|
|01/01/2007| 447292|
|      null|6418484|
|01/01/2008| 425068|
|01/01/2018| 120884|
|01/01/2014| 279712|
|01/01/2005| 459528|
|01/01/2015| 266472|
|01/01/2003| 489758|
|01/01/2012| 346588|
|01/01/2010| 382404|
|01/01/2009| 393972|
|01/01/2017| 270630|
|01/01/2016| 279158|
+----------+-------+



## Quantidade de crimes por ano que sejam do tipo NARCOTICS?
---

#### Criando uma View temporaria em memoria com o dataframe 

In [11]:
df.createOrReplaceTempView("ocorrencias")

#### Usando SQL para afzer as querys 

* Primeiro defini meu target(ano)
* Depois passei um count(*) para pegar todas as ocorrencias 
* Como Condicional passei o tipo da ocorrencias NARCOTICS 
* Agrupando por Ano e Ordenando por Ano

In [12]:
CrimesPorAnoNarc = spark.sql("SELECT  Ano, COUNT(*) as ocorrencias FROM ocorrencias WHERE Tipo = 'NARCOTICS' GROUP BY Ano ORDER BY Ano")

In [13]:
CrimesPorAnoNarc.show()

+----+-----------+
| Ano|ocorrencias|
+----+-----------+
|2001|      50996|
|2002|      50086|
|2003|      55342|
|2004|      58500|
|2005|      57266|
|2006|      55966|
|2007|      55936|
|2008|      45876|
|2009|      42708|
|2010|      44344|
|2011|      38916|
|2012|      36490|
|2013|      34778|
|2014|      29752|
|2015|      21986|
|2016|      13988|
|2017|      11902|
|2018|       5760|
+----+-----------+



## Quantidade de crimes por ano que sejam do tipo NARCOTICS e tenham ocorrido em dias pares?
---
* Primeiro defini meu target(ano)
* Depois passei um count(*) para pegar todas as ocorrencias 
* Como Condicional passei o tipo da ocorrencias NARCOTICS 
* Agrupando por Ano e Ordenando por Ano
* Passei o boolean type AND com o Parametro Dia % 2 == 0


In [14]:
sql = """SELECT Ano, COUNT(*) as ocorrencias FROM ocorrencias WHERE Tipo = 'NARCOTICS' AND (Dia % 2) == 0  GROUP BY Ano  ORDER BY Ano """                

CrimesPorAnoNarcParDay = spark.sql(sql)

In [15]:
CrimesPorAnoNarcParDay.show()

+----+-----------+
| Ano|ocorrencias|
+----+-----------+
|2001|      24732|
|2002|      24610|
|2003|      26726|
|2004|      28296|
|2005|      27610|
|2006|      27648|
|2007|      27736|
|2008|      22302|
|2009|      20894|
|2010|      21632|
|2011|      19330|
|2012|      17884|
|2013|      16822|
|2014|      14644|
|2015|      10746|
|2016|       6690|
|2017|       5732|
|2018|       2678|
+----+-----------+



## Mês com maior ocorrência de crimes?
----

Passei o Parametro Mes dentro da subquery contando o numero de linhas dos Mes e chamando de ocorrencias com isso agrupei por Mes e ordenei para apresentar de ordem descendente, limitando apenas 1 unico valor de saida

In [16]:
sql = "select Mes from (select Mes, count(Mes) as ocorrencias from ocorrencias group by Mes order by 'ocorrencias' desc) LIMIT 1"

MesMaiorOcorrencia = spark.sql(sql)
MesMaiorOcorrencia.show()


+---+
|Mes|
+---+
|  7|
+---+



## Mês com a maior média de ocorrência de crimes?
----

Aqui fiz 4 querys 

* 1 - Para agrupar por mes a media score das ocorrencias 
* 2 - A segunda pega o maximo valor dessa avg_ocorrencias 
* 3 - Faz um select de mes, AVG(total_linhas) agrupando por Mes 
* 4 - Ultima query ele pega os Valores Maximos de Mes, e Avg_ocorre, usando condicional where para passar a segunda query com condição para avg_ocorre = MAX valor da média

In [17]:
sum_count = df.count() 

In [18]:
print(sum_count)

13157184


In [19]:
sql = "SELECT Mes, avg_ocorre  FROM (SELECT Mes, AVG(13157184) AS avg_ocorre FROM ocorrencias GROUP BY Mes)   WHERE avg_ocorre = (SELECT MAX(avg_ocorre) FROM (SELECT Mes, AVG(13157184) AS avg_ocorre FROM ocorrencias GROUP BY Mes)) LIMIT 1"
spark.sql(sql).show()

+---+-----------+
|Mes| avg_ocorre|
+---+-----------+
|  7|1.3157184E7|
+---+-----------+



## Mês por ano com a maior ocorrência de crimes?

---

* Aqui passei um groubBy(mes) para grupar por mes
* Em Seguida passei um agregador agg para o valor Max do Ano
* Limitando apenas uma unica saida 

In [20]:
df.groupBy("Mes").agg(F.max("Ano")).limit(1).show()

+---+--------+
|Mes|max(Ano)|
+---+--------+
|  7|    2017|
+---+--------+



## Mês com a maior ocorrência de crimes do tipo DECEPTIVE PRACTICE?
---

Aqui pensei que poderia rodar um test checando pelo pyspark e tambem com sql para validar as duas saidas


A Primeira DF_DECEPTIVE_PRACTICE passei um filter para pegar apenas os tipos DECEPTIVE PRACTICE 

Em Seguida agrupei Mes e Ano usando agregação para contar o as vezes que os dados da coluna mes aparece chamando de Mes_ocorr

In [21]:
DF_DECEPTIVE_PRACTICE = df.filter(df['Tipo']  == "DECEPTIVE PRACTICE")
DF_DECEPTIVE_PRACTICE = DF_DECEPTIVE_PRACTICE.groupby(['Mes', 'Ano']).agg(F.count("Mes").alias("Mes_Ocorr"))

Com a DF criado passei os parametros MAX em todas as dimenções de atenção da tabela(Mes, Ano, Mes_Ocorr)

In [22]:
value = DF_DECEPTIVE_PRACTICE.agg({"Mes_Ocorr": "max","Mes":"max","Ano":"Max"}).collect()
print(value)

[Row(max(Ano)='2018', max(Mes_Ocorr)=1824, max(Mes)='9')]


### Validação com SQL

Com o dado do dataframe pensei em passar pelo sql para validar a saida dos dados pela query a baixo

Passei um MAX(mes, Mes_, Ano) e em uma subquery passei os parametros e Mes,Ano e Count(Mes)

Usando WHERE passei o tipo do dado buscado e agrupei por Mes e Ano

In [23]:
sql = """
select max(Mes) as Mes_, max(Ano) as Ano_, max(Mes_ocorr) from
(select Mes,Ano,count(Mes) as Mes_ocorr FROM ocorrencias WHERE Tipo = 'DECEPTIVE PRACTICE' GROUP BY Mes, Ano)

"""
spark.sql(sql).show()

+----+----+--------------+
|Mes_|Ano_|max(Mes_ocorr)|
+----+----+--------------+
|   9|2018|          1824|
+----+----+--------------+



## Dia do ano com a maior ocorrência de crimes?

----

PS: Por falha e erro jo job não consegui rodar essa query

Contudo a ideia era agrupar Mes, Ano e Dia e ordenar por ocorrencias e com isso fazer um count do mes

In [25]:
sql = """
  SELECT Mes, Ano, Dia  FROM ( 
        SELECT Mes, 
        COUNT(Mes) as ocorrencias, 
        Ano,
        Dia  
        FROM ocorrencias  GROUP BY Mes, Ano, Dia 
        ORDER BY 'ocorrencias' DESC) LIMIT 1
"""
spark.sql(sql).show()

    

Py4JJavaError: An error occurred while calling o84.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 15 in stage 40.0 failed 1 times, most recent failure: Lost task 15.0 in stage 40.0 (TID 1718, sandbox-host.hortonworks.com, executor driver): java.io.FileNotFoundException: /tmp/blockmgr-ff9c78ab-fdb4-43b1-846f-2de9dc87e1a6/14/temp_shuffle_4c9dfea7-cb4a-4cad-872b-5bbaf1d38b7d (Too many open files)
	at java.io.FileOutputStream.open0(Native Method)
	at java.io.FileOutputStream.open(FileOutputStream.java:270)
	at java.io.FileOutputStream.<init>(FileOutputStream.java:213)
	at org.apache.spark.storage.DiskBlockObjectWriter.initialize(DiskBlockObjectWriter.scala:105)
	at org.apache.spark.storage.DiskBlockObjectWriter.open(DiskBlockObjectWriter.scala:118)
	at org.apache.spark.storage.DiskBlockObjectWriter.write(DiskBlockObjectWriter.scala:245)
	at org.apache.spark.shuffle.sort.BypassMergeSortShuffleWriter.write(BypassMergeSortShuffleWriter.java:158)
	at org.